<a href="https://colab.research.google.com/github/AmirGhnbr/smart_beta_portfolio_optimization/blob/main/smart_beta_portfolio_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Smart Beta Portfolio Optimization

## Overview of the project

In [1]:
!pip install cvxpy yfinance

In [2]:
import pandas as pd
import numpy as np
import cvxpy as cp
import yfinance as yf

In [3]:
import yfinance as yf

tickers = ['AMZN', 'AAPL', 'MSFT', 'TSLA','GOOGL']
start_date = '2020-01-01'
end_date = '2020-12-31'

data = yf.download(tickers, start=start_date, end=end_date)
data.head()

[*********************100%***********************]  5 of 5 completed


Adj Close                                                   Close  \
                 AAPL       AMZN      GOOGL        MSFT       TSLA       AAPL   
Date                                                                            
2020-01-02  73.347939  94.900497  68.433998  155.422043  28.684000  75.087502   
2020-01-03  72.634857  93.748497  68.075996  153.486755  29.534000  74.357498   
2020-01-06  73.213608  95.143997  69.890503  153.883499  30.102667  74.949997   
2020-01-07  72.869286  95.343002  69.755501  152.480438  31.270666  74.597504   
2020-01-08  74.041496  94.598503  70.251999  154.909225  32.809334  75.797501   

                                                         ...       Open  \
                 AMZN      GOOGL        MSFT       TSLA  ...       AAPL   
Date                                                     ...              
2020-01-02  94.900497  68.433998  160.619995  28.684000  ...  74.059998   
2020-01-03  93.748497  68.075996  158.619995  29.534000  ...  74.287498   
2020-01-06  95.143997  69.890503  159.029999  30.102667  ...  73.447502   
2020-01-07  95.343002  69.755501  157.580002  31.270666  ...  74.959999   
2020-01-08  94.598503  70.251999  160.089996  32.809334  ...  74.290001   

                                                            Volume            \
                 AMZN      GOOGL        MSFT       TSLA       AAPL      AMZN   
Date                                                                           
2020-01-02  93.750000  67.420502  158.779999  28.299999  135480400  80580000   
2020-01-03  93.224998  67.400002  158.320007  29.366667  146322800  75288000   
2020-01-06  93.000000  67.581497  157.080002  29.364668  118387200  81236000   
2020-01-07  95.224998  70.023003  159.320007  30.760000  108872000  80898000   
2020-01-08  94.902000  69.740997  158.929993  31.580000  132079200  70160000   

                                           
               GOOGL      MSFT       TSLA  
Date                                       
2020-01-02  27278000  22622100  142981500  
2020-01-03  23408000  21116200  266677500  
2020-01-06  46768000  20813700  151995000  
2020-01-07  34330000  21634100  268231500  
2020-01-08  35314000  27746500  467164500  

[5 rows x 30 columns]

In [4]:
close = data['Adj Close']
volume = data['Volume']

## Part 1

implement market cap index weighted.


The index we'll be using is based on large dollar volume stocks.

In [5]:
def generate_dollar_volume_weights(close, volume):
    """
    Generate dollar volume weights.

    Parameters
    ----------
    close : DataFrame
        Close price for each ticker and date
    volume : str
        Volume for each ticker and date

    Returns
    -------
    dollar_volume_weights : DataFrame
        The dollar volume weights for each ticker and date
    """
    market_cap = close * volume
    total_market_cap_date = market_cap.sum(axis=1)
    dollar_volume_weights = market_cap.divide(total_market_cap_date,axis='rows')

    return dollar_volume_weights

In [6]:
market_cap_index_weight = generate_dollar_volume_weights(close, volume)
market_cap_index_weight

,AAPL,AMZN,GOOGL,MSFT,TSLA
Date,,,,,
2020-01-02,0.367116,0.282511,0.068964,0.129893,0.151516
2020-01-03,0.349645,0.232199,0.052424,0.106625,0.259107
2020-01-06,0.315831,0.281636,0.119104,0.116708,0.166722
2020-01-07,0.266869,0.259457,0.080555,0.110966,0.282153
2020-01-08,0.253859,0.172288,0.064400,0.111575,0.397878
...,...,...,...,...,...
2020-12-23,0.250140,0.146550,0.043623,0.088809,0.470878
2020-12-24,0.238188,0.153619,0.026927,0.076641,0.504625
2020-12-28,0.265018,0.295208,0.038768,0.062362,0.338643


## Returns
implementing `get_returns` function the generate return data for all the stocks from the prices dataframe.

In [7]:
def get_returns(prices):
  '''
  Generate returns for ticker and date.

  Parameters
  ----------
  prices : DataFrame
      Price for each ticker and date

  Returns
  -------
  returns : Dataframe
      The returns for each ticker and date

  '''

  return (prices - prices.shift(1))/prices.shift(1)

In [8]:
returns = get_returns(close)
returns.head(10)

,AAPL,AMZN,GOOGL,MSFT,TSLA
Date,,,,,
2020-01-02,NaN,NaN,NaN,NaN,NaN
2020-01-03,-0.009722,-0.012139,-0.005231,-0.012452,0.029633
2020-01-06,0.007968,0.014886,0.026654,0.002585,0.019255
2020-01-07,-0.004703,0.002092,-0.001932,-0.009118,0.038801
2020-01-08,0.016086,-0.007809,0.007118,0.015929,0.049205
2020-01-09,0.021241,0.004799,0.010498,0.012493,-0.021945
2020-01-10,0.002261,-0.009411,0.006459,-0.004627,-0.006627
2020-01-13,0.021364,0.004323,0.007747,0.012024,0.097689
2020-01-14,-0.013503,-0.011558,-0.006555,-0.007043,0.024883


## Weighted Returns
With the returns of each stock computed, we can use it to compute the returns for an index or ETF. Implement `generate_weighted_returns` to create weighted returns using the returns and weights.

In [9]:
def generate_weighted_returns(returns, weights):
    """
    Generate weighted returns.

    Parameters
    ----------
    returns : DataFrame
        Returns for each ticker and date
    weights : DataFrame
        Weights for each ticker and date

    Returns
    -------
    weighted_returns : DataFrame
        Weighted returns for each ticker and date
    """
    weighted_returns = returns*weights

    return weighted_returns

In [10]:
index_weighted_return = generate_weighted_returns(returns, market_cap_index_weight)
index_weighted_return.head(10)

,AAPL,AMZN,GOOGL,MSFT,TSLA
Date,,,,,
2020-01-02,NaN,NaN,NaN,NaN,NaN
2020-01-03,-0.003399,-0.002819,-0.000274,-0.001328,0.007678
2020-01-06,0.002517,0.004192,0.003175,0.000302,0.003210
2020-01-07,-0.001255,0.000543,-0.000156,-0.001012,0.010948
2020-01-08,0.004084,-0.001345,0.000458,0.001777,0.019577
2020-01-09,0.007136,0.000755,0.000646,0.001095,-0.007847
2020-01-10,0.000881,-0.001850,0.000443,-0.000548,-0.001502
2020-01-13,0.005874,0.000664,0.000501,0.001201,0.039739
2020-01-14,-0.004181,-0.001861,-0.000306,-0.000649,0.009716


## Cumulative Returns
the `get_cumulative_return` function calculates the cumulative returns over the given time period of the returns.

In [11]:
def get_cumulative_returns(returns):
    """
    Calculate cumulative returns.

    Parameters
    ----------
    returns : DataFrame
        Returns for each ticker and date

    Returns
    -------
    cumulative_returns : Pandas Series
        Cumulative returns for each date
    """
    #TODO: Implement function

    return (returns.sum(axis = 1) + 1).cumprod()

In [12]:
index_weighted_cumulative_returns = get_cumulative_returns(index_weighted_return)
index_weighted_cumulative_returns

Date
2020-01-02    1.000000
2020-01-03    0.999858
2020-01-06    1.013252
2020-01-07    1.022440
2020-01-08    1.047542
                ...   
2020-12-23    4.695163
2020-12-24    4.762104
2020-12-28    4.868408
2020-12-29    4.868100
2020-12-30    4.949698
Length: 252, dtype: float64

## Part 2: Portfolio Optimization


We want to both minimize the portfolio variance and also want to closely track a market cap weighted index. In other words, we're trying to minimize the distance between the weights of our portfolio and the weights of the index.

## Covariance

Implement `get_covariance_returns` to calculate the covariance of the returns. We'll use this to calculate the portfolio variance.


In [13]:
def get_covariance_returns(returns):
    """
    Calculate covariance matrices.

    Parameters
    ----------
    returns : DataFrame
        Returns for each ticker and date

    Returns
    -------
    returns_covariance  : 2 dimensional Ndarray
        The covariance of the returns
    """
    cleaned_returns = returns.fillna(0)
    returns_covariance = np.cov(cleaned_returns.T)

    return returns_covariance

In [14]:
covariance_returns = get_covariance_returns(returns)


In [15]:
covariance_returns = pd.DataFrame(covariance_returns, returns.columns, returns.columns)

covariance_returns_correlation = np.linalg.inv(np.diag(np.sqrt(np.diag(covariance_returns))))
covariance_returns_correlation = pd.DataFrame(
    covariance_returns_correlation.dot(covariance_returns).dot(covariance_returns_correlation),
    covariance_returns.index,
    covariance_returns.columns)

In [18]:
covariance_returns_correlation.head()

,AAPL,AMZN,GOOGL,MSFT,TSLA
AAPL,1.000000,0.697065,0.752484,0.839575,0.472519
AMZN,0.697065,1.000000,0.675757,0.738936,0.459282
GOOGL,0.752484,0.675757,1.000000,0.858142,0.446808
MSFT,0.839575,0.738936,0.858142,1.000000,0.504955
TSLA,0.472519,0.459282,0.446808,0.504955,1.000000


## Portfolio variance

we can write the portfolio variance in quadratic form, using cvxpy `quad_form(x,p)` function .

### Objective function
We want to minimize both the portfolio variance and the distance of the portfolio weights from the index weights.

### Constraints
We can also define our constraints in a list. For example, you'd want the weights to sum to one.

### optimization
So now that we have our objective function and constraints, we can solve for the values of `X` . cvxpy has the constructor `Problem(objective, constraints)`, which returns a Problem object.

In [19]:
import cvxpy as cvx

def get_optimal_weights(covariance_returns, index_weights, scale=2.0):
    """
    Find the optimal weights.

    Parameters
    ----------
    covariance_returns : 2 dimensional Ndarray
        The covariance of the returns
    index_weights : Pandas Series
        Index weights for all tickers at a period in time
    scale : int
        The penalty factor for weights the deviate from the index
    Returns
    -------
    x : 1 dimensional Ndarray
        The solution for x
    """
    m = covariance_returns.shape[0]

    x = cvx.Variable(m)

    portfolio_variance = cvx.quad_form(x, covariance_returns)

    distance_to_index = cvx.norm(x-index_weights)

    objective = cvx.Minimize(portfolio_variance + scale * distance_to_index)

    constraints = [x >= 0, sum(x) == 1]

    problem = cvx.Problem(objective, constraints)
    min_value = problem.solve()

    x_values = x.value

    return x_values

## Optimized Portfolio
Using the `get_optimal_weights` function, let's generate the optimal ETF weights without rebalanceing. We can do this by feeding in the covariance of the entire history of data.

In [24]:
raw_optimal_single_rebalance_etf_weights = get_optimal_weights(covariance_returns.values, market_cap_index_weight.iloc[-1])
optimal_single_rebalance_etf_weights = pd.DataFrame(
    np.tile(raw_optimal_single_rebalance_etf_weights, (len(returns.index), 1)),
    returns.index,
    returns.columns)

In [25]:
optimal_single_rebalance_etf_weights.head(10)

,AAPL,AMZN,GOOGL,MSFT,TSLA
Date,,,,,
2020-01-02,0.214505,0.178005,0.030812,0.074181,0.502496
2020-01-03,0.214505,0.178005,0.030812,0.074181,0.502496
2020-01-06,0.214505,0.178005,0.030812,0.074181,0.502496
2020-01-07,0.214505,0.178005,0.030812,0.074181,0.502496
2020-01-08,0.214505,0.178005,0.030812,0.074181,0.502496
2020-01-09,0.214505,0.178005,0.030812,0.074181,0.502496
2020-01-10,0.214505,0.178005,0.030812,0.074181,0.502496
2020-01-13,0.214505,0.178005,0.030812,0.074181,0.502496
2020-01-14,0.214505,0.178005,0.030812,0.074181,0.502496


In [29]:
optim_etf_returns = generate_weighted_returns(returns, optimal_single_rebalance_etf_weights)
optim_etf_returns.head(10)

,AAPL,AMZN,GOOGL,MSFT,TSLA
Date,,,,,
2020-01-02,NaN,NaN,NaN,NaN,NaN
2020-01-03,-0.002085,-0.002161,-0.000161,-0.000924,0.014891
2020-01-06,0.001709,0.002650,0.000821,0.000192,0.009675
2020-01-07,-0.001009,0.000372,-0.000060,-0.000676,0.019497
2020-01-08,0.003451,-0.001390,0.000219,0.001182,0.024725
2020-01-09,0.004556,0.000854,0.000323,0.000927,-0.011027
2020-01-10,0.000485,-0.001675,0.000199,-0.000343,-0.003330
2020-01-13,0.004583,0.000769,0.000239,0.000892,0.049088
2020-01-14,-0.002897,-0.002057,-0.000202,-0.000522,0.012504
